In [1]:
import pandas as pd
import plotly.express as px
import sqlite3

pd.set_option('display.max_rows', None)

### Most-Played Cards / Card Winrates

In [8]:
conn = sqlite3.connect("../mtgmelee.db")
query = """
WITH full_results as (
SELECT
  player1_id,
  player1_deck_id,
  winner_id
FROM results
WHERE player2_id IS NOT NULL

UNION ALL
SELECT
  player2_id as player1_id,
  player2_deck_id as player1_deck_id,
  winner_id
FROM results
WHERE player2_id IS NOT NULL
),

results_summary as (
SELECT
  d.tournament_id,
  d.id,
  d.player_id,
  SUM(CASE WHEN r.winner_id = r.player1_id THEN 1 ELSE 0 END) as wins,
  SUM(CASE WHEN r.winner_id != r.player1_id THEN 1 ELSE 0 END) as losses,
  SUM(CASE WHEN r.winner_id = r.player1_id THEN 1 ELSE 0 END)  * 1.0 / COUNT(*) as win_pct  
FROM decks d
JOIN full_results r
  on d.id = r.player1_deck_id
GROUP BY 1, 2, 3
ORDER BY 1, 4 DESC , 6 DESC
)

SELECT
  r.tournament_id,
  c.name,
  SUM(wins) as wins,
  SUM(losses) as losses,
  SUM(quantity) as num_copies_main,
  ROUND(SUM(wins) * 100.0 / SUM(wins + losses)) as win_pct
FROM results_summary r
JOIN cards c
  ON r.id = c.deck_id
  AND c.is_sideboard = 0
GROUP BY 1, 2
HAVING
  SUM(wins + losses) > 10
ORDER BY 1, SUM(quantity) DESC
"""

df = pd.read_sql_query(query, conn)
tournaments = list(set(df['tournament_id']))
tournaments.sort()

for tournament in tournaments:
    df_t = df[df['tournament_id']==tournament]
    fig = px.scatter(df_t, x='num_copies_main', y='win_pct',
                     hover_data=['name'], title=f'Most Played Cards & Winrates: Tournament {tournament}',
                     labels=dict(num_copies_main='Number of Copies (in All Main Decks)',
                                 win_pct='Win %')
                    )
    fig.show()



### Player Winrates

In [3]:
conn = sqlite3.connect("../mtgmelee.db")
query = """
WITH full_results as (
SELECT
  player1_id,
  player1_deck_id,
  winner_id
FROM results
WHERE player2_id IS NOT NULL

UNION ALL
SELECT
  player2_id as player1_id,
  player2_deck_id as player1_deck_id,
  winner_id
FROM results
WHERE player2_id IS NOT NULL
)

SELECT
    player1_id as player,
    p.name as name,
    SUM(CASE WHEN winner_id = player1_id THEN 1 ELSE 0 END) as wins,
    SUM(CASE WHEN winner_id != player1_id THEN 1 ELSE 0 END) as losses,
    COUNT(*) as matches_played,
    ROUND(SUM(CASE WHEN winner_id = player1_id THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as win_pct
FROM full_results r
JOIN players p
    ON r.player1_id = p.id
GROUP BY 1, 2
ORDER BY 3 DESC
"""

df = pd.read_sql_query(query, conn)
df



,player,name,wins,losses,matches_played,win_pct
0,Juggler314,Jack Kiefer,50,26,76,66.0
1,zach_dubin,Zach Dubin,48,34,82,59.0
2,bparis15,Bryan Paris,44,23,67,66.0
3,CeciliaJupe,Ty Thomason,43,19,62,69.0
4,TESpacito,Jamie Miller,43,33,76,57.0
5,TombSimon,Simon Nielsen,42,29,71,59.0
6,BanjoCargo,Jacob Nagro,40,21,61,66.0
7,fabfoe,Fabian Thiele,38,33,71,54.0
8,Valorj,Joshua Bausch,37,28,65,57.0
9,Worldwaker2,Philip Steudel,37,24,61,61.0


### Most-Played Cards in Top-8s

In [4]:
conn = sqlite3.connect("../mtgmelee.db")
query = """
WITH top8_results as (
SELECT
    player1_deck_id as deck_id
FROM results
WHERE
    rnd NOT LIKE 'Round%'
    
UNION ALL
SELECT
    player2_deck_id as deck_id
FROM results
WHERE
    rnd NOT LIKE 'Round%'
),

top8_decks as (
SELECT
    distinct deck_id
FROM top8_results)

SELECT
    d.tournament_id,
    c.name,
    SUM(quantity) as quantity
FROM top8_decks r
JOIN decks d
    ON r.deck_id = d.id
JOIN cards c
    ON r.deck_id = c.deck_id
    AND c.is_sideboard = 0
GROUP BY 1, 2
ORDER BY 1, 3 DESC
"""

df = pd.read_sql_query(query, conn)
df



,tournament_id,name,quantity
0,389,Swamp,86
1,389,Dark Ritual,23
2,389,Graveyard Marshal,20
3,389,Gravecrawler,20
4,389,Geralf's Messenger,20
5,389,Diregraf Ghoul,20
6,389,Death Baron,19
7,389,Lord of the Undead,15
8,389,Plains,14
9,389,Polluted Delta,12
